In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

from qiskit import IBMQ, BasicAer, Aer, QuantumCircuit, execute
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit import BasicAer, Aer, IBMQ
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.initial_states import Zero
from qiskit.aqua.components.optimizers import COBYLA, L_BFGS_B, SLSQP, SPSA
from qiskit.aqua.operators import WeightedPauliOperator, Z2Symmetries
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock

from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import QuantumError, ReadoutError
from qiskit.providers.aer.noise.errors import pauli_error
from qiskit.providers.aer.noise.errors import depolarizing_error
from qiskit.providers.aer.noise.errors import thermal_relaxation_error

from qiskit.providers.aer import noise
provider = IBMQ.load_account()

import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from qiskit.quantum_info.states.random import random_statevector
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.utils import get_subsystem_density_matrix
from qiskit.aqua import aqua_globals
from scipy.special import rel_entr
from qiskit.extensions import CXGate
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit.providers.aer.extensions.snapshot_statevector import *
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# setup qiskit.chemistry logging
import logging
from qiskit.chemistry import set_qiskit_chemistry_logging
set_qiskit_chemistry_logging(logging.ERROR)

/opt/conda/lib/python3.8/site-packages/qiskit/chemistry/__init__.py:170: DeprecationWarning: The package qiskit.chemistry is deprecated. It was moved/refactored to qiskit_nature (pip install qiskit-nature). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('chemistry', 'qiskit_nature', 'qiskit-nature')


In [2]:
backend = Aer.get_backend("qasm_simulator")
print(backend)

qasm_simulator


In [3]:
#Set the seed for reproducible results
aqua_globals.random_seed = 100
seed_simulator = 100
seed_transpiler=100
np.random.seed(100)

In [4]:
#Alternative way to attain the Haar distribution
def haar_distribution():
    nplist = np.load('haar_distribution.npy')
    flist = nplist.tolist()
    if len(flist) == 0:
        print('the list is empty')
        flist = []
        for m in range(5000):
            a = random_statevector(4)
            b = random_statevector(4)
            flist.append(state_fidelity(a, b))
        np.save('haar_distribution.npy', flist)
    return flist


def get_pqc(params,ansatz):
    param_dict={}
    for i in range(ansatz.num_parameters):
        param_dict[list(ansatz.ordered_parameters)[i]]=params[i]

    #print(param_dict)
    pqc=ansatz.bind_parameters(param_dict)
    return pqc


def get_circuit_distribution(ansatz):
    fidelityList = []
    for m in range(5000):
        qstatelist =[]
        for i in range(2):
            # Obtain a quantum circuit instance from the paramters
            params = np.random.random(ansatz.num_parameters)*2*np.pi
            qc = get_pqc(params,ansatz)
            result = execute(qc, backend, seed_simulator=seed_simulator, seed_transpiler=seed_transpiler).result()
            #Qasm simulator
            outputstate = result.data(0)['snapshots']['statevector']['final']
            qstatelist.append(outputstate[0])
        fidelityList.append(state_fidelity(qstatelist[0], qstatelist[1]))
    return fidelityList

In [5]:
#Compute KL divergence between distributions of state fidelities
def kl_divergence(p):
    N=16
    num_bins = 75
    bin_width = 1/num_bins
    bin_edges = np.linspace(0,1,num_bins+1)
    
    chist, _ =  np.histogram(np.array(p), bins=bin_edges, density= True)
    #Use analytical form to attain probability distribution of fidelities for ensemble of Haar random states
    fhist = np.array([(N-1)*(1-F)**(N-2) for F in bin_edges[:-1]])
    
    kl_pq = np.sum(rel_entr(chist*bin_width, fhist/sum(fhist)))
    print('KL(P || Q): %.3f ' % kl_pq)
    return kl_pq

In [ ]:
#hlist = haar_distribution()
num_qubits = 4
var_forms = [('RY_CX', 'ry', 'cx', [(2, 3), (1, 2), (0, 1), (0, 3)]),
             ('RY_CZ', 'ry', 'cz', [(2, 3), (1, 2), (0, 1), (0, 3)]),
             ('RYRZ_CX', ['ry', 'rz'], 'cx', 'full'),
             ('RYRZ_CZ', ['ry', 'rz'], 'cz','full'),
             ('HRX_CX', ['h', 'rx'], 'cx', 'full'),
             ('HRX_CZ', ['h', 'rx'], 'cz', 'full')]
# CX or CZ circuit experiment
for k in range(len(var_forms)):
    print(var_forms[k])
    ansatz = TwoLocal(num_qubits, var_forms[k][1], var_forms[k][2], reps=1, entanglement=var_forms[k][3], insert_barriers=True)
    #ansatz = TwoLocal(4, 'ry', 'cz', reps=1, entanglement=[(2, 3), (1, 2), (0,1), (0,3)], insert_barriers=True)
    #ansatz = TwoLocal(num_qubits, ['ry', 'rz'], 'cx', reps= 1,entanglement= 'full' , insert_barriers=True)
    #ansatz = TwoLocal(num_qubits, ['ry', 'rz'], 'cz', reps=1, entanglement= 'full', insert_barriers=True)
    #ansatz = TwoLocal(num_qubits, ['h', 'rx'], 'cx', entanglement='full', reps=1, parameter_prefix = 'y',insert_barriers=True)
    #ansatz = TwoLocal(num_qubits, ['h', 'rx'], 'cz', entanglement=[(2, 3), (1, 2), (0,1), (0,3)], reps=1, insert_barriers=True)
    circuit = ansatz.snapshot_statevector('final')
    fidelityList = get_circuit_distribution(ansatz)
    kl = kl_divergence(fidelityList)

('RY_CX', 'ry', 'cx', [(2, 3), (1, 2), (0, 1), (0, 3)])
KL(P || Q): 0.224 
('RY_CZ', 'ry', 'cz', [(2, 3), (1, 2), (0, 1), (0, 3)])
KL(P || Q): 0.226 
('RYRZ_CX', ['ry', 'rz'], 'cx', 'full')
KL(P || Q): 0.020 
('RYRZ_CZ', ['ry', 'rz'], 'cz', 'full')
KL(P || Q): 0.027 
('HRX_CX', ['h', 'rx'], 'cx', 'full')
